In [8]:
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
from gensim.models import KeyedVectors
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import numpy as np
from collections import OrderedDict

from utils import *





In [3]:
wiki_300 = KeyedVectors.load_word2vec_format('~/gensim-data/glove-wiki-gigaword-300/glove-wiki-gigaword-300.txt', binary=False)

In [4]:
topic = 'international organized crime'

similar_words = expand_query(topic, wiki_300, 30).split()[3:]
similar_words

['organised',
 'criminal',
 'murder',
 'trafficking',
 'global',
 'terrorism',
 'murders',
 'gang',
 'violence',
 'sponsored',
 'world',
 'criminals',
 'participated',
 'homicide',
 'groups',
 'corruption',
 'activities',
 'gangs',
 'mafia',
 'violent',
 'robbery',
 'participating',
 'enforcement',
 'hate',
 'nations',
 'killings',
 'drug',
 'based',
 'staged',
 'prostitution']

# Relevance model

In [5]:
hits = Searcher.search(topic, 10)
ids = []
for h in hits:
    ids.append(h.docid)

ids

['FBIS4-40260',
 'FBIS4-46734',
 'FBIS3-24145',
 'FBIS4-39330',
 'FBIS4-7811',
 'FBIS3-23986',
 'FBIS3-37418',
 'FBIS4-41991',
 'FBIS4-16951',
 'FBIS4-22471']

In [10]:
def merge_doc_vectors(docs):
    tfs = dict()
    for d in docs:
        tf = IndexReader.reader.get_document_vector(d)
        for k in tf:
            if k in tfs:
                tfs[k] += tf[k]
            else:
                tfs[k] = tf[k]
    return tfs

doc_vectors = merge_doc_vectors(ids)


In [37]:


def rank_words(doc_vectors, words):
    top_words = {}
    for w in words:
        if w in doc_vectors:
            if w in top_words:
                top_words[w] += doc_vectors[w]
            else:
                top_words[w] = doc_vectors[w]
    ordered_dict = dict(sorted(top_words.items(), key=lambda item: item[1], reverse=True))
    return list(ordered_dict.keys())

    
def expand_query_using_relevance_feedback(model, topic, n = None, top_docs = 10):
    if n is None:
        n = len(topic.split())
    hits = Searcher.search(topic, top_docs)
    doc_ids = [h.docid for h in hits]
    doc_vector = merge_doc_vectors(doc_ids)
    top_words = []
    itr = 1
    while len(top_words) <= n:
        prev_len = len(top_words)
        new_words = expand_query(topic, model, 10*n*itr).split()[n:]
        potentials = rank_words(doc_vector, new_words)
        top_words += potentials[(itr-1)*n:itr*n]
        itr+=1
        if prev_len == len(top_words) or itr >= 10:
            break
        
    return top_words[:n]
        
        
        
    

expand_query_using_relevance_feedback(wiki_300, topic)

        

['drug', 'world', 'murder']